In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import sqlite3
import joblib
import os
import json

# --- Налаштування шляхів для збереження ---
MODELS_DIR = 'models'
LOGS_DIR = 'logs'
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

# --- Зчитування даних ---
database_file = 'data/cleaned-data.db'
table_name = 'Main Table'

conn = sqlite3.connect(database_file)
tables_query = "SELECT * FROM 'Main Table';"
cleaned_df = pd.read_sql_query(tables_query, conn)
conn.close()

# --- Підготовка даних ---
def prepare_data(df):
    # Створення цільової змінної
    conditions = [
        df['charger_type_Level 1'] == True,
        df['charger_type_Level 2'] == True
    ]
    choices = ['Level 1', 'Level 2']
    df['ChargerType_Target'] = np.select(conditions, choices, default='DC Fast Charger')
    
    # Вибір ознак і цільової змінної
    y_column = 'ChargerType_Target'

    columns_to_drop = [
        'charging_station_id',
        'charging_start_time',
        'charging_end_time',
        'vehicle_model',
        'charging_station_location',
        'time_of_day',
        'day_of_week',
        'charger_type_Level 1',
        'charger_type_Level 2',
        'ChargerType_Target'
    ]
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    x_columns = df.columns.drop(existing_columns_to_drop)
    
    X = df[x_columns]
    y = df[y_column]
    
    # Локальна нормалізація ознак для k-NN
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return X, y

# Підготовка даних
X, y = prepare_data(cleaned_df)

# --- Ініціалізація моделей ---
models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVC": SVC(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5)
}

# --- Ініціалізація StratifiedKFold ---
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# --- Тренування моделей із StratifiedKFold ---
model_training_logs = []

for name, model in models.items():
    print(f"\nТренування моделі: {name}")
    
    fold_logs = []  # Логи для кожного фолду
    
    # Виконання StratifiedKFold
    for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
        print(f"  Fold {fold}...")
        
        # Розбиття на тренувальну та валідаційну вибірки
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        # Тренування моделі на тренувальних даних фолду
        model.fit(X_train, y_train)
        
        # Логування результатів для поточного фолду
        fold_logs.append({
            'fold': fold,
            'status': 'completed',
            'training_timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        })
        print(f"  Fold {fold} завершено.")
    
    # Додавання загального логу для моделі
    model_training_logs.append({
        'model': name,
        'status': 'completed',
        'training_timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'fold_logs': fold_logs
    })
    print(f"Модель {name} натренована.")

# --- Збереження логів у JSON ---
training_log_path = os.path.join(LOGS_DIR, 'training_logs.json')
with open(training_log_path, 'w') as :
    json.dump(model_training_logs, f, indent=4)


SyntaxError: invalid syntax (3364304133.py, line 118)